## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.98,69,5,7.52,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,44.71,70,0,8.05,clear sky
2,2,Hilo,US,19.7297,-155.0900,73.80,90,0,0.00,clear sky
3,3,Kahramanmaras,TR,38.0000,37.0000,21.83,100,97,4.03,overcast clouds
4,4,Ketchikan,US,55.3422,-131.6461,45.07,70,40,9.22,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.98,69,5,7.52,clear sky
12,12,Fuerte Olimpo,PY,-21.0415,-57.8738,79.47,65,21,6.15,few clouds
14,14,Esperance,AU,-33.8667,121.9000,83.80,46,3,16.28,clear sky
17,17,Atuona,PF,-9.8000,-139.0333,78.78,77,20,16.40,few clouds
29,29,Sambava,MG,-14.2667,50.1667,77.50,77,100,4.23,overcast clouds
33,33,Tubuala,PA,9.5167,-79.0333,77.32,82,10,14.27,clear sky
43,43,Buala,SB,-8.1450,159.5921,84.16,77,100,9.26,light rain
59,59,Saint-Philippe,RE,-21.3585,55.7679,87.39,89,100,24.61,moderate rain
60,60,Kapaa,US,22.0752,-159.3190,75.52,77,20,5.75,few clouds
65,65,Buluang,PH,13.3068,123.3425,82.29,69,96,10.47,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                427
City                   427
Country                426
Lat                    427
Lng                    427
Max Temp               427
Humidity               427
Cloudiness             427
Wind Speed             427
Current Description    427
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                426
City                   426
Country                426
Lat                    426
Lng                    426
Max Temp               426
Humidity               426
Cloudiness             426
Wind Speed             426
Current Description    426
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.98,clear sky,-23.1203,-134.9692,
12,Fuerte Olimpo,PY,79.47,few clouds,-21.0415,-57.8738,
14,Esperance,AU,83.80,clear sky,-33.8667,121.9000,
17,Atuona,PF,78.78,few clouds,-9.8000,-139.0333,
29,Sambava,MG,77.50,overcast clouds,-14.2667,50.1667,
33,Tubuala,PA,77.32,clear sky,9.5167,-79.0333,
43,Buala,SB,84.16,light rain,-8.1450,159.5921,
59,Saint-Philippe,RE,87.39,moderate rain,-21.3585,55.7679,
60,Kapaa,US,75.52,few clouds,22.0752,-159.3190,
65,Buluang,PH,82.29,overcast clouds,13.3068,123.3425,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))